## 1. Import

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random



In [15]:
import importlib
import connect_four
import random_bot
import logic_bot
importlib.reload(connect_four)
importlib.reload(random_bot)
importlib.reload(logic_bot)
from connect_four import ConnectFour
from random_bot import RandomBot
from logic_bot import LogicBot

## 2. NN Bot Class

In [16]:
import nn_bot
importlib.reload(nn_bot)
from nn_bot import NeuralNetworkBot


## Functions

In [4]:
def initialize_population(population_size, piece):
    return [NeuralNetworkBot(piece) for _ in range(population_size)]

In [5]:
def evaluate_fitness(bot, opponents, games=10):
    fitness = 0
    opponent = random.choice(opponents)
    for _ in range(games):
        game = ConnectFour()
        while game.state == 'UNFINISHED':
            col = bot.choose_column(game)
            game.insert(col, bot.piece)
            if game.state != 'UNFINISHED':
                fitness += 1 if game.state == bot.piece + '_WON' else 0
                fitness += -1 if game.state == 'DRAW' else 0
                break

            col = opponent.choose_column(game)
            game.insert(col, opponent.piece)
            if game.state != 'UNFINISHED':
                fitness += -2 if game.state == opponent.piece + '_WON' else 0
                fitness += -1 if game.state == 'DRAW' else 0
                break
    return fitness

In [6]:
def select_top_performers(population, fitnesses, num_top):
    sorted_indices = np.argsort(fitnesses)[::-1]
    return [population[i] for i in sorted_indices[:num_top]]

In [7]:
def generate_new_population(top_performers, population_size, mutation_rate=0.01):
    new_population = []
    while len(new_population) < population_size:
        if len(top_performers) > 1:
            parent1, parent2 = random.sample(top_performers, 2)
        else:
            parent1 = parent2 = top_performers[0]
        child = parent1.crossover(parent2)
        child.mutate(mutation_rate)
        new_population.append(child)
    return new_population

In [8]:
# def train_evolutionary(population_size=50, generations=100, num_top=10, mutation_rate=0.01, games_per_fitness=10):
#     population = initialize_population(population_size, 'X')
#     opponent = RandomBot('O')

#     for generation in range(generations):
#         fitnesses = [evaluate_fitness(bot, opponent, games_per_fitness) for bot in population]
#         top_performers = select_top_performers(population, fitnesses, num_top)
#         population = generate_new_population(top_performers, population_size, mutation_rate)
#         print(f'Generation {generation + 1}, Best Fitness: {max(fitnesses)}')

#     return population
def train_evolutionary(population_size=50, generations=100, games_per_fitness=10, num_top=10, mutation_rate=0.01, opponent=None):
    # Initialize population with random bots
    populations = []
    population = [NeuralNetworkBot('X') for _ in range(population_size)]
    if not opponent:
        opponent = RandomBot('O')
    previous_top_performers = None
    populations.append(population)

    for generation in range(generations):
        fitnesses = [evaluate_fitness(bot, [opponent], games_per_fitness) for bot in population]

        if previous_top_performers:
            fitnesses_prev = [evaluate_fitness(bot, previous_top_performers, games_per_fitness // 2) for bot in population]
            fitnesses = [f1 + f2 for f1, f2 in zip(fitnesses, fitnesses_prev)]

        top_performers = select_top_performers(population, fitnesses, num_top)
        previous_top_performers = top_performers

        population = generate_new_population(top_performers, population_size, mutation_rate)
        populations.append(population)
        print(f"Generation {generation}: Best fitness = {max(fitnesses)}")

    return populations

In [9]:
# parallel version
from concurrent.futures import ProcessPoolExecutor

def evaluate_fitness_parallel(args):
    return evaluate_fitness(*args)

def train_parallel_evolutionary(population_size=20, generations=100, games_per_fitness=20, num_top=5, mutation_rate=0.05, opponent=None):
    population = [NeuralNetworkBot('X') for _ in range(population_size)]
    if not opponent:
        opponent = RandomBot('O')
    previous_top_performers = None
    best_fitness_over_generations = []

    with ProcessPoolExecutor() as executor:
        for generation in range(generations):
            fitness_args = [(bot, opponent, games_per_fitness) for bot in population]
            fitnesses = list(executor.map(evaluate_fitness_parallel, fitness_args))

            if previous_top_performers:
                fitness_args_prev = [(bot, previous_top_performers, games_per_fitness // 2) for bot in population]
                fitnesses_prev = list(executor.map(evaluate_fitness_parallel, fitness_args_prev))
                fitnesses = [f1 + f2 for f1, f2 in zip(fitnesses, fitnesses_prev)]

            top_performers = select_top_performers(population, fitnesses, num_top)
            previous_top_performers = top_performers
            population = generate_new_population(top_performers, population_size, mutation_rate)
            best_fitness = max(fitnesses)
            best_fitness_over_generations.append(best_fitness)
            print(f"Generation {generation}: Best fitness = {best_fitness}")

    return population, best_fitness_over_generations

In [22]:
model_path = r"C:\Users\storm\Documents\GitHub\Connect4\Connect-4-Game\models\2024-05-25-13-15\gen10_model15.keras"
NeuralNetworkBot('O', tf.keras.models.load_model(model_path))
trained_populations = train_evolutionary(population_size=50, generations=10, games_per_fitness=20, num_top=10, mutation_rate=0.01, opponent=LogicBot('O'))

Generation 0: Best fitness = -25
Generation 1: Best fitness = -18
Generation 2: Best fitness = -9
Generation 3: Best fitness = -12
Generation 4: Best fitness = -12
Generation 5: Best fitness = -12
Generation 6: Best fitness = -12
Generation 7: Best fitness = 0
Generation 8: Best fitness = -12
Generation 9: Best fitness = -6


In [13]:
trained_populations = train_parallel_evolutionary(population_size=50, generations=5, games_per_fitness=20, num_top=10, mutation_rate=0.01, opponent=LogicBot('O'))

c:\Users\storm\Documents\GitHub\Connect4\Connect-4-Game\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
import os
from datetime import datetime

now = datetime.now()
dir_name = os.path.join('models', now.strftime("%Y-%m-%d-%H-%M"))

os.makedirs(dir_name, exist_ok=True)
for generation, population in enumerate(trained_populations):
    for ind, bot in enumerate(population):
        model_path = os.path.join(dir_name, f'gen{generation}_model{ind}.keras')
        bot.model.save(model_path)
